In [1]:
import pandas as pd
from nsepy import get_history as gh
import datetime as dt
import yfinance as yf

In [2]:
import datetime as dt

start = dt.datetime(2021, 7, 1)
end = dt.datetime(2022, 1, 27)
symbol = 'HDFCBANK.NS'  # Stock symbol in uppercase

stk_data = yf.download(symbol, start=start, end=end)

[*********************100%%**********************]  1 of 1 completed


In [3]:
stk_data=stk_data[["Open","High","Low","Close"]]
dataset=stk_data

In [4]:
from sklearn.preprocessing import MinMaxScaler
Ms = MinMaxScaler()
data1= Ms.fit_transform(stk_data)
print("Len:",data1.shape)

Len: (143, 4)


In [5]:
data1=pd.DataFrame(data1,columns=["Open","High","Low","Close"])


In [6]:
training_size = round(len(data1 ) * 0.80)
print(training_size)
X_train=data1[:training_size]
X_test=data1[training_size:]
print("X_train length:",X_train.shape)
print("X_test length:",X_test.shape)
y_train=data1[:training_size]
y_test=data1[training_size:]
print("y_train length:",y_train.shape)
print("y_test length:",y_test.shape)

114
X_train length: (114, 4)
X_test length: (29, 4)
y_train length: (114, 4)
y_test length: (29, 4)


In [7]:
import warnings
warnings.filterwarnings("ignore")

In [8]:
performance={"Model":[],"RMSE":[],"MaPe":[],"Lag":[],"Test":[]}


In [16]:
def cominbation(dataset,listt):
    print(listt)
    datasetTwo=dataset[listt]
    test_obs = 28
    train =datasetTwo[:-test_obs]
    test = datasetTwo[-test_obs:]
    from statsmodels.tsa.api import VAR
    for i in [1,2,3,4,5,6,7,8,9,10]:
        model = VAR(train)
        results = model.fit(i)
        print('Order =', i)
        print('AIC: ', results.aic)
        print('BIC: ', results.bic)
        print()
    x = model.select_order(maxlags=12)
    order=x.selected_orders["aic"]
    result = model.fit(order)
    #result.summary()
    lagged_Values = train.values[-order:]
    pred = result.forecast(y=lagged_Values,steps=28) 
    preds=pd.DataFrame(pred,columns=listt)
    preds.to_csv("varforecasted_{}.csv".format(test_obs))
    from sklearn.metrics import mean_squared_error
    rmse= round(mean_squared_error(test,pred,squared=False))
    from sklearn.metrics import mean_absolute_percentage_error
    mape=mean_absolute_percentage_error(test,pred)
    performance["Model"].append(listt)
    performance["RMSE"].append(rmse)
    performance["MaPe"].append(mape)
    performance["Lag"].append(order)
    performance["Test"].append(test_obs)
    perf=pd.DataFrame(performance)
    return perf,result,pred

In [17]:
listt=["Close","High","Open","Low"]


In [18]:
perf,result,pred=cominbation(data1,listt)


['Close', 'High', 'Open', 'Low']
Order = 1
AIC:  -27.053142343626618
BIC:  -26.57310752811881

Order = 2
AIC:  -26.89590836009499
BIC:  -26.027006046168932

Order = 3
AIC:  -26.75798442186105
BIC:  -25.495824231616897

Order = 4
AIC:  -26.569280971348824
BIC:  -24.909388595770096

Order = 5
AIC:  -26.52986124111685
BIC:  -24.467676234511735

Order = 6
AIC:  -26.415860157286374
BIC:  -23.946733659828446

Order = 7
AIC:  -26.232535002144715
BIC:  -23.35172738782611

Order = 8
AIC:  -26.003180031894214
BIC:  -22.705858479100087

Order = 9
AIC:  -25.902020909220454
BIC:  -22.183256891026247

Order = 10
AIC:  -25.71514913152432
BIC:  -21.569915822706854



In [19]:
perf


,Model,RMSE,MaPe,Lag,Test
0,"[Close, High, Open, Low]",0,0.958963,1,28
1,"[Close, High, Open, Low]",0,0.958963,1,28
